# Arvak + Qrisp Integration

This notebook demonstrates seamless integration between Arvak and Qrisp, enabling:
- Bi-directional circuit conversion (Qrisp ↔ Arvak)
- Using Arvak compilation with Qrisp circuits
- Leveraging Qrisp's high-level quantum programming model
- Automatic uncomputation and quantum data structures

Qrisp is a high-level quantum programming framework that emphasizes:
- **QuantumVariable**: High-level quantum data structures
- **Automatic uncomputation**: Efficient resource management
- **QuantumSession**: Compile and manage quantum programs
- **Built-in algorithms**: Ready-to-use quantum algorithms

## Installation

```bash
pip install arvak[qrisp]
```

## Step 1: Check Integration Status

First, let's verify that the Qrisp integration is available.

In [ ]:
import arvak

# Verify Qrisp integration is available
status = arvak.integration_status()
print("Available integrations:")
for name, info in status.items():
    icon = "✓" if info['available'] else "✗"
    print(f"  {icon} {name}: {info['packages']}")

# Check specifically for Qrisp
if 'qrisp' not in status or not status['qrisp']['available']:
    raise ImportError(
        "Qrisp integration not available. "
        "Install with: pip install qrisp>=0.4.0"
    )

print("\n✓ Qrisp integration is available!")

## Step 2: Create Circuit with Qrisp's QuantumCircuit

Let's start with Qrisp's lower-level QuantumCircuit API (similar to Qiskit).

In [ ]:
from qrisp import QuantumCircuit

# Create a Bell state circuit in Qrisp
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
for q in qc.qubits:
    qc.measure(q)

print("Qrisp QuantumCircuit:")
print(qc)

## Step 3: Convert Qrisp Circuit to Arvak

Convert the Qrisp circuit to Arvak format using the integration API.

In [ ]:
# Get Qrisp integration
qrisp_integration = arvak.get_integration('qrisp')

# Convert to Arvak
arvak_circuit = qrisp_integration.to_arvak(qc)

print("Arvak Circuit:")
print(f"  Name: {arvak_circuit.name}")
print(f"  Qubits: {arvak_circuit.num_qubits}")
print(f"  Classical bits: {arvak_circuit.num_clbits}")
print(f"  Depth: {arvak_circuit.depth()}")
print(f"  Gate count: {arvak_circuit.size()}")

## Step 4: High-Level Programming with QuantumVariable

Qrisp's unique feature is high-level quantum programming with QuantumVariable.

In [ ]:
from qrisp import QuantumVariable, h, cx

# Create a QuantumVariable (high-level quantum register)
qv = QuantumVariable(2)  # 2 qubits

# Apply gates using high-level API
h(qv[0])  # Hadamard on first qubit
cx(qv[0], qv[1])  # CNOT

print("QuantumVariable Circuit:")
print(qv.qs)  # QuantumSession

## Step 5: Convert QuantumSession to Arvak

Qrisp's QuantumSession can also be converted to Arvak.

In [ ]:
# Get the compiled circuit from QuantumSession
qrisp_compiled = qv.qs.compile()

# Convert to Arvak
arvak_from_qv = qrisp_integration.to_arvak(qrisp_compiled)

print("Arvak Circuit from QuantumVariable:")
print(f"  Qubits: {arvak_from_qv.num_qubits}")
print(f"  Depth: {arvak_from_qv.depth()}")
print("\nOpenQASM representation:")
print(arvak.to_qasm(arvak_from_qv))

## Step 6: Convert Arvak Circuit Back to Qrisp

In [ ]:
# Create a circuit in Arvak
arvak_original = arvak.Circuit.ghz(3)

# Convert to Qrisp
qrisp_circuit = qrisp_integration.from_arvak(arvak_original)

print("Converted to Qrisp:")
print(qrisp_circuit)

## Step 7: Configure Arvak Compilation for Hardware

Use Arvak's advanced compilation capabilities with Qrisp circuits.

In [ ]:
from arvak import CouplingMap, BasisGates, PropertySet, Layout

# Configure for IQM hardware
coupling_map = CouplingMap.star(5)
basis_gates = BasisGates.iqm()
layout = Layout.trivial(5)

props = PropertySet().with_target(coupling_map, basis_gates)
props.set_layout(layout)

print("IQM Hardware Configuration:")
print(f"  Topology: star with 5 qubits")
print(f"  Edges: {coupling_map.edges()}")
print(f"  Native gates: {basis_gates.gates()}")

## Step 8: Backend Comparison

In [ ]:
# Compare different hardware configurations
backends_config = [
    ("IQM", BasisGates.iqm(), CouplingMap.star(5)),
    ("IBM", BasisGates.ibm(), CouplingMap.linear(5)),
    ("Simulator", BasisGates.universal(), CouplingMap.full(5))
]

print("Backend Comparison:")
print("=" * 70)
for name, gates, topology in backends_config:
    print(f"\n{name}:")
    print(f"  Native gates: {gates.gates()}")
    print(f"  Connectivity: {topology.edges()}")
    print(f"  Qubits: {topology.num_qubits}")

## Step 9: Use Arvak Backend with Qrisp

Execute Qrisp circuits through Arvak's backend.

Runs on Arvak's built-in statevector simulator (up to ~20 qubits). The simulator executes entirely in Rust via PyO3 — no network calls, no mocks.

In [ ]:
from arvak.integrations.qrisp import ArvakBackendClient

# Get Arvak backend for Qrisp
backend = ArvakBackendClient('sim')
print(f"Backend: {backend.name}")

# Execute Qrisp circuit
results = backend.run(qc, shots=1000)

print("\nExecution Results:")
for bitstring, count in sorted(results.items()):
    bar = "█" * int(count / 10)
    print(f"  {bitstring}: {count:4d} {bar}")

## Step 10: Qrisp's Automatic Uncomputation

One of Qrisp's key features is automatic uncomputation, which helps manage ancilla qubits efficiently.

In [ ]:
from qrisp import QuantumBool, mcx

# Create quantum booleans
a = QuantumBool()
b = QuantumBool()

# Set initial states
a[:] = True  # Set to |1⟩
b[:] = False  # Set to |0⟩

# Compute XOR with automatic uncomputation
result = a ^ b  # XOR operation

print("Qrisp Automatic Uncomputation Example:")
print(f"  a = True, b = False")
print(f"  a XOR b = result")
print(f"\nCircuit depth: {result.qs.depth()}")
print("\nQrisp automatically manages ancilla qubits and uncomputes!")

## Step 11: Complex Example - Quantum Phase Estimation with Qrisp

In [ ]:
from qrisp import QuantumFloat

# Create a QuantumFloat for phase estimation
# This is a high-level quantum type that Qrisp provides
qf = QuantumFloat(3, -3)  # 3 integer bits, -3 exponent (precision)

print("QuantumFloat for Phase Estimation:")
print(f"  Integer bits: 3")
print(f"  Precision: 2^-3 = 0.125")
print(f"  Total qubits: {qf.size}")

# Get the underlying circuit
circuit_from_qf = qf.qs.compile()
print(f"\nUnderlying circuit qubits: {circuit_from_qf.num_qubits()}")

## Step 12: Export for CLI Execution

In [ ]:
# Convert Qrisp circuit to Arvak format
arvak_export = qrisp_integration.to_arvak(qc)

# Export to QASM for CLI execution
output_qasm = arvak.to_qasm(arvak_export)

# Save to file
with open("qrisp_circuit.qasm", "w") as f:
    f.write(output_qasm)

print("Circuit exported to qrisp_circuit.qasm")
print("\nTo execute:")
print("  $ arvak run qrisp_circuit.qasm --backend sim --shots 1000")
print("\nWith specific backend:")
print("  $ arvak run qrisp_circuit.qasm --backend iqm --shots 1000")

## Summary

This notebook demonstrated:

1. **Integration Status**: Checking if Qrisp integration is available
2. **Circuit Creation**: Using Qrisp's QuantumCircuit API
3. **Conversion to Arvak**: Converting Qrisp circuits to Arvak format
4. **High-Level Programming**: Using QuantumVariable and QuantumSession
5. **Round-Trip Conversion**: Arvak ↔ Qrisp circuit conversion
6. **Hardware Configuration**: Configuring Arvak compilation for specific backends
7. **Backend Execution**: Running circuits through Arvak backend
8. **Automatic Uncomputation**: Qrisp's unique resource management
9. **Quantum Data Types**: QuantumBool, QuantumFloat, etc.
10. **Export for Production**: Saving circuits for CLI execution

## Key Advantages of Qrisp + Arvak

- **High-Level + Optimized**: Qrisp's high-level programming with Arvak's compilation
- **Automatic Uncomputation**: Qrisp manages resources, Arvak optimizes hardware
- **Hardware Agnostic**: Write once in Qrisp, compile for any backend with Arvak
- **Algorithm Library**: Use Qrisp's built-in algorithms with Arvak's backends

## Next Steps

- Explore **01_core_arvak.ipynb** for pure Arvak programming
- Explore **02_qiskit_integration.ipynb** for Qiskit integration
- Explore **04_cirq_integration.ipynb** for Cirq integration
- Use the Arvak CLI for production execution on real hardware
- Check out the Arvak dashboard for interactive development

## Resources

- Arvak Documentation: https://github.com/hiq-lab/HIQ
- Qrisp Documentation: https://qrisp.eu/
- Qrisp GitHub: https://github.com/eclipse-qrisp/Qrisp
- OpenQASM 3.0 Spec: https://openqasm.com/